<a href="https://colab.research.google.com/github/Yashu2699/Deep_learning/blob/main/6_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import keras
import math
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Layer
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Input, concatenate, GlobalAveragePooling2D, AveragePooling2D, Flatten
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

In [ ]:
num_classes = 10

(x_train, y_train), (x_valid, y_valid) = cifar10.load_data()

  #resizing
x_train = np.array([cv2.resize(img, (224, 224)) for img in x_train[:,:,:,:]])
x_valid = np.array([cv2.resize(img, (224, 224)) for img in x_valid[:,:,:,:]])

y_train = np_utils.to_categorical(y_train, num_classes)
y_valid = np_utils.to_categorical(y_valid, num_classes)

x_train = x_train.astype('float32') / 255.0
x_valid = x_valid.astype('float32') / 255.0

  

In [ ]:
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj, name=None):
  conv_1x1 = Conv2D(filters_1x1, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_3x3 = Conv2D(filters_3x3_reduce, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_3x3 = Conv2D(filters_3x3, (3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)
  conv_5x5 = Conv2D(filters_5x5_reduce, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
  conv_5x5 = Conv2D(filters_5x5, (5,5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)
  pool_proj = MaxPool2D((3,3), strides=(1,1), padding='same')(x)
  pool_proj = Conv2D(filters_pool_proj, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

  output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
  return output

In [ ]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.constant(value=0.2)

In [ ]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7,7), padding='same', strides=(2,2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3,3), padding='same', strides=(2,2), name='max_pool_1_3x3/2')(x)
x = Conv2D(192, (3,3), padding='same', strides=(1,1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3,3), padding='same', strides=(2,2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,  64, 96, 128, 16, 32, 32, name='inception_3a')
x = inception_module(x, 128, 128, 192, 32, 96, 64, name='inception_3b')

x = MaxPool2D((3,3), padding='same', strides=(2,2), name='max_pool_3x3/2')(x)

x = inception_module(x, 192, 96, 208, 16, 48, 64, name='inception_4a')

classifier1 = AveragePooling2D((5,5), strides=3)(x)
classifier1 = Conv2D(128, (1,1), padding='same', activation='relu')(classifier1)
classifier1 = Flatten()(classifier1)
classifier1 = Dense(1024, activation='relu')(classifier1)
classifier1 = Dropout(0.7)(classifier1)
classifier1 = Dense(10, activation='softmax', name='aucilliary_output_1')(classifier1)

x = inception_module(x, 160, 112, 224, 24, 64, 64, name='inception_4b')
x = inception_module(x, 128, 128, 256, 24, 64, 64, name='inception_4c')
x = inception_module(x, 112, 144, 288, 32, 64, 64, name='inception_4d')

classifier2 = AveragePooling2D((5,5), strides=3)(x)
classifier2 = Conv2D(128, (1,1), padding='same', activation='relu')(classifier2)
classifier2 = Flatten()(classifier2)
classifier2 = Dense(1024, activation='relu')(classifier2)
classifier2 = Dropout(0.7)(classifier2)
classifier2 = Dense(10, activation='softmax', name='auxilliary_output_2')(classifier2)

x = inception_module(x, 256, 160, 320, 32, 128, 128, name='inception_4e')

x = MaxPool2D((3,3), padding='same', strides=(2,2), name='max_pool_4_3x3/2')(x)

x = inception_module(x, 256, 160, 320, 32, 128, 128, name='inception_5a')
x = inception_module(x, 384, 192, 384, 48, 128, 128, name='inception_5b')

x = AveragePooling2D((7,7), strides=1, padding='valid', name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)
x = Dense(1000, activation='relu', name='linear')(x)
x = Dense(1000, activation='softmax', name='output')(x)

In [ ]:
model = Model(input_layer, [x], name='googlenet')

In [ ]:
model.summary()

Model: "googlenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv_1_7x7/2 (Conv2D)          (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pool_1_3x3/2 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv_1_7x7/2[0][0]']           
 )                                                                                        

In [ ]:
model_with_classifiers = Model(input_layer, [x, classifier1, classifier2], name='googlenet_complete_architecture')
model_with_classifiers.summary()

Model: "googlenet_complete_architecture"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv_1_7x7/2 (Conv2D)          (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pool_1_3x3/2 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv_1_7x7/2[0][0]']           
 )                                                                  

In [ ]:
epochs = 25
initial_lrate = 0.01
def decay(epoch, steps=100):
  initial_lrate=0.01
  drop=0.96
  epochs_drop=8
  lrate = initial_lrate*math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

sgd = SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)
lr_sc = LearningRateScheduler(decay, verbose=1)
model_with_classifiers.compile(loss=['categorical_crossentropoy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1,0.3,0.3], optimizer=sgd, metrics=['accuracy'])
history = model_with_classifiers.fit(x_train, [y_train, y_train, y_train], epochs=epochs, validation_data=(x_test, [y_test, y_test, y_test]), batch_size=256, callbacks=[lr_sc])